# Extracting location information from tweets <br>

_Author: Bala Krishnamoorthy_

In [104]:
import pandas as pd
import re

In [105]:
tweets_df = pd.read_csv('../data/tweets_with_intersection.csv');

In [ ]:
tweets_df.drop(axis=1, columns='Unnamed: 0', inplace=True)

In [119]:
tweets_df.head()

,text,intersection,coordinates.coordinates,created_at,place.bounding_box.coordinates,place.country,place.country_code,place.full_name,user.created_at,user.location
0,"Road construction, left lane closed in #Albuqu...",Tijeras Ave E,"[-106.65, 35.08653]",Mon Jan 14 14:56:55 +0000 2019,"[[[-106.7916912, 35.0158912], [-106.473745, 35...",United States,US,"Albuquerque, NM",Tue Feb 08 23:44:59 +0000 2011,Albuquerque
1,Road construction. right lanes closed in #Pima...,I-10 EB at Ruthrauff Rd,"[-111.0295, 32.29418]",Mon Jan 14 14:31:30 +0000 2019,"[[[-111.083219, 32.057802], [-110.747928, 32.0...",United States,US,"Tucson, AZ",Thu Feb 10 00:00:33 +0000 2011,"Tucson, AZ"
2,"Road construction, shoulder closed in #ElPaso ...",I 10 Both EB/WB from Executive Ctr Blvd to Sun...,"[-106.52, 31.79295]",Mon Jan 14 14:28:12 +0000 2019,"[[[-106.634874, 31.6206683], [-106.199987, 31....",United States,US,"El Paso, TX",Thu Feb 10 20:42:54 +0000 2011,"El Paso, TX"
3,Ughhh at the dentist for a cleaning and the si...,NaN,NaN,Mon Jan 14 14:08:05 +0000 2019,"[[[-86.044756, 41.426657], [-85.9629436, 41.42...",United States,US,"Nappanee, IN",Thu Jul 30 19:27:12 +0000 2009,"New Paris, IN"
4,Road constructions. two right lanes closed in ...,I-10 EB at Ruthrauff Rd,"[-111.0295, 32.29418]",Mon Jan 14 13:49:35 +0000 2019,"[[[-111.083219, 32.057802], [-110.747928, 32.0...",United States,US,"Tucson, AZ",Thu Feb 10 00:00:33 +0000 2011,"Tucson, AZ"
5,All eastbound lanes are closed due to snow and...,Angeles Crest Hwy EB between CA-39 and Big Pin...,"[-117.8511, 34.35686]",Mon Jan 14 13:13:00 +0000 2019,"[[[-124.482003, 32.528832], [-114.131212, 32.5...",United States,US,"California, USA",Wed Aug 12 21:43:18 +0000 2009,Southern California


In [108]:
tweets_df_2 = tweets_df[['text', 'place.bounding_box.coordinates', 'place.full_name']]
tweets_df_2.head()

,text,place.bounding_box.coordinates,place.full_name
0,"Road construction, left lane closed in #Albuqu...","[[[-106.7916912, 35.0158912], [-106.473745, 35...","Albuquerque, NM"
1,Road construction. right lanes closed in #Pima...,"[[[-111.083219, 32.057802], [-110.747928, 32.0...","Tucson, AZ"
2,"Road construction, shoulder closed in #ElPaso ...","[[[-106.634874, 31.6206683], [-106.199987, 31....","El Paso, TX"
3,Ughhh at the dentist for a cleaning and the si...,"[[[-86.044756, 41.426657], [-85.9629436, 41.42...","Nappanee, IN"
4,Road constructions. two right lanes closed in ...,"[[[-111.083219, 32.057802], [-110.747928, 32.0...","Tucson, AZ"


In [109]:
# Number of tweets by location 
tweets_df['place.full_name'].value_counts()[:5]

Fort Worth, TX     13
Albuquerque, NM    11
San Antonio, TX     7
Phoenix, AZ         6
El Paso, TX         5
Name: place.full_name, dtype: int64

Fort Worth has the most number of tweets about road blockages. Examine tweets from this city. 

In [28]:
# Fort Worth, TX Tweets as a Series
tweets_text_fw = tweets_df[tweets_df['place.full_name'] == 'Fort Worth, TX'][['text']]
tweets_text_fw.reset_index(inplace=True, drop=True)
tweets_text_fw.head()
for i in tweets_text_fw['text']:
    print(i)

Road construction. two left lanes closed in #FortWorth on 35W SB between Morningside Dr and E Felix St #DFWTraffic https://t.co/4NzBppXRvr
Closed due to road construction in #FortWorth on 35W SB at Pharr, stop and go traffic back to Northside Dr… https://t.co/AyfxrFRbqU
Closed due to road construction in #FortWorth on 35W SB at Pharr #DFWTraffic https://t.co/4NzBppXRvr
Road construction. two left lanes closed in #FortWorth on 35W SB between Berry St and Seminary Dr #DFWTraffic https://t.co/4NzBppXRvr
Road construction. three left lanes closed in #FortWorth on 35W SB between Berry St and Seminary Dr #DFWTraffic https://t.co/4NzBppXRvr
Road construction. three left lanes closed in #FortWorth on 35W SB between Morningside Dr and Seminary Dr… https://t.co/mSqz9axgJX
Road construction. three left lanes closed in #FortWorth on 35W SB between Allen Ave and Seminary Dr, stop and go... https://t.co/4NzBppXRvr
Road construction. two left lanes closed in #FortWorth on 35W SB between Allen Ave and

Part 1 - Filter tweets to only use data on closed roads:

In [92]:
# Create new column
tweets_df['closed_road_check'] = ''
# Enter 1 if tweet indicates 'closed road', 0 if not
closed_road_indicators = ['Closed due to', 'Road closed']
index = 0
for i in tweets_df['text']:
    if any(x in i for x in closed_road_indicators):
        tweets_df['closed_road_check'][index] = 1
    else:
        tweets_df['closed_road_check'][index] = 0

/Users/Bala_K/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/Bala_K/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [94]:
tweets_df['closed_road_check']

0      0
1       
2       
3       
4       
5       
6       
7       
8       
9       
10      
11      
12      
13      
14      
15      
16      
17      
18      
19      
20      
21      
22      
23      
24      
25      
26      
27      
28      
29      
      ..
113     
114     
115     
116     
117     
118     
119     
120     
121     
122     
123     
124     
125     
126     
127     
128     
129     
130     
131     
132     
133     
134     
135     
136     
137     
138     
139     
140     
141     
142     
Name: closed_road_check, Length: 143, dtype: object

### Filtered Tweets Below (Only Closed Roads)

In [110]:
tweets_classed_df = pd.read_csv('../data/2-interim/twitter_corpus.csv', sep='\t', 
                                encoding='latin-1')
tweets_classed_df.drop(axis=1, columns='tweed_id', inplace=True)

In [ ]:
tweets_classed_df.head()

In [129]:
# 2 = closed
final_tweets_df = tweets_classed_df[tweets_classed_df['rating'] == 2]
final_tweets_df.head()

,rating,tweet
5,2,All eastbound lanes are closed due to snow and...
10,2,The drainage project on Center Street in #Vine...
11,2,Closed due to road construction in #FortWorth ...
12,2,Closed due to road construction in #FortWorth ...
15,2,Closed due to road construction in #Evergreen ...


In [130]:
# Combine with original df (join by index) to pull city info
final_tweets_df = pd.merge(final_tweets_df, tweets_df, left_index=True, 
                           right_index=True)

In [134]:
final_tweets_df = final_tweets_df[['tweet','created_at',
                                   'place.bounding_box.coordinates','place.full_name']]
final_tweets_df.reset_index(drop=True, inplace=True)

In [135]:
final_tweets_df.head()

,tweet,created_at,place.bounding_box.coordinates,place.full_name
0,All eastbound lanes are closed due to snow and...,Mon Jan 14 13:13:00 +0000 2019,"[[[-124.482003, 32.528832], [-114.131212, 32.5...","California, USA"
1,The drainage project on Center Street in #Vine...,Mon Jan 14 11:23:01 +0000 2019,"[[[-70.62084, 41.4245617], [-70.586828, 41.424...","Vineyard Haven, MA"
2,Closed due to road construction in #FortWorth ...,Mon Jan 14 10:33:20 +0000 2019,"[[[-97.538285, 32.569477], [-97.033542, 32.569...","Fort Worth, TX"
3,Closed due to road construction in #FortWorth ...,Mon Jan 14 10:32:16 +0000 2019,"[[[-97.538285, 32.569477], [-97.033542, 32.569...","Fort Worth, TX"
4,Closed due to road construction in #Evergreen ...,Mon Jan 14 04:43:56 +0000 2019,"[[[-109.060257, 36.992427], [-102.041524, 36.9...","Colorado, USA"


In [136]:
final_tweets_df['place.full_name'].value_counts()

Phoenix, AZ            6
Houston, TX            4
California, USA        4
Oak Ridge North, TX    3
San Antonio, TX        3
South Valley, NM       2
Fort Worth, TX         2
Mobile, AL             2
Albuquerque, NM        2
Arizona, USA           2
Revere, MA             1
Buffalo, NY            1
Pennsylvania, USA      1
Indiana, USA           1
Golden Glades, FL      1
Montgomery, AL         1
Silver Spring, MD      1
Tuscaloosa, AL         1
Ohio, USA              1
Brooklyn, NY           1
Colorado, USA          1
Thousand Oaks, CA      1
Seattle, WA            1
Norland, FL            1
Coral Springs, FL      1
Suffield, CT           1
Marysville, WA         1
Massachusetts, USA     1
Cambridge, MA          1
Nipomo, CA             1
Davie, FL              1
Oxford, AL             1
Vineyard Haven, MA     1
Irondale, AL           1
Alabama, USA           1
Appleton, WI           1
Charlotte, NC          1
Fair Oaks Ranch, TX    1
Name: place.full_name, dtype: int64

Regex Step 2 - Extract street information:

2.1: Isolate street names and intersections

#### Choose City in Code Block Below

In [246]:
city = 'Houston, TX'
city_tweets = final_tweets_df[final_tweets_df['place.full_name'] == city]['tweet']
city_tweets.reset_index(drop=True, inplace=True)
intersection_list = []
bbox

In [278]:
# Extract bounding box (i.e. bbox) for chosen city -- Needed to extract HereMaps data for same
# city
bbox_raw = eval(final_tweets_df[final_tweets_df['place.full_name'] == city]\
['place.bounding_box.coordinates'][6])[0]

# twitter bbox is always defined by 4 corners of the bbox
bbox_top_left = str(bbox_raw[3][1]) + ',' + str(bbox_raw[3][0])
bbox_bottom_right = str(bbox_raw[1][1]) + ',' + str(bbox_raw[1][0])
print('bbox_top_left (lat,long):', bbox_top_left)
print('bbox_bottom_right (lat,long):', bbox_bottom_right)

bbox_top_left (lat,long): 30.1546646,-95.823268
bbox_bottom_right (lat,long): 29.522325,-95.069705


In [247]:
# Generate string of "useful" tweets
for i in range(city_tweets.shape[0]):
    # Regex Pattern 1
    intersection_list.append(re.findall(' on (.+)[#]', city_tweets[i]))
    if i == (city_tweets.shape[0] - 1):
        print('intersection list:', intersection_list)
        print()
        print('number of tweets available:', city_tweets.shape[0])
        print('number of patterns found:', len(intersection_list))

intersection list: [['288 S Fwy NB at The S Lp, stopped traffic back to Reed Rd. '], ['S Sam Houston Tollway EB between Hwy 288 and Cullen '], ['S Sam Houston Tollway EB between Hwy 288 and Cullen '], ['S Sam Houston Tollway EB between Hwy 288 and Cullen ']]

number of tweets available: 4
number of patterns found: 4


In [178]:
# Convert list of lists to list of strings
intersection_list_2 = []

for i in intersection_list:
    intersection_list_2.append(i[0])

print('updated intersection list:', intersection_list_2)

updated intersection list: ['288 S Fwy NB at The S Lp, stopped traffic back to Reed Rd. ', 'S Sam Houston Tollway EB between Hwy 288 and Cullen ', 'S Sam Houston Tollway EB between Hwy 288 and Cullen ', 'S Sam Houston Tollway EB between Hwy 288 and Cullen ']


Regex 2.2: Use text data to create start and end points

In [211]:
temp = intersection_list_2[0].split('at')
temp_2 = temp[]
temp_2 = temp[0] + '&' + temp_

['288 S Fwy NB ', ' The S Lp, stopped traffic back to Reed Rd. ']

In [226]:
# Instantiate df to hold output intersections
columns = ['city', 'start', 'end', 'tweet']
output_tweets_df = pd.DataFrame(index=range(len(city_tweets)), columns=columns)
output_tweets_df.fillna('', inplace=True)
output_tweets_df

,city,start,end,tweet
0,,,,
1,,,,
2,,,,
3,,,,


In [227]:
output_tweets_df.loc[:, 'city'] = city
output_tweets_df

,city,start,end,tweet
0,"Houston, TX",,,
1,"Houston, TX",,,
2,"Houston, TX",,,
3,"Houston, TX",,,


In [228]:
index = 0
for string in intersection_list_2:
    print(string)
    output_tweets_df.loc[index, 'tweet'] = city_tweets[index]
    # Pattern 1
    if 'between' in string:
        split_1 = string.split('between') # Split on 'between' first
        split_2 = split_1[1].split('and')
        start = split_1[0] + '&' + split_2[0]
        end = split_1[0] + '&' + split_2[1]
        output_tweets_df.loc[index, 'start'] = start
        output_tweets_df.loc[index, 'end'] = end
        print('start:', start)
        print('end:', end)
    # Pattern 2
    elif 'at' in string:
        split_1 = string.split('at') # Split on 'at' first
        split_2 = split_1[1].split(',')
        start = split_1[0] + '& ' + split_2[0]
        end = split_1[0] + '& ' + split_2[1]
        output_tweets_df.loc[index, 'start'] = start
        output_tweets_df.loc[index, 'end'] = end
        print('start:', start)
        print('end:', end)
    else:
        print('no matching patterns found for tweet at index:')
    index += 1
    print()

288 S Fwy NB at The S Lp, stopped traffic back to Reed Rd. 
start: 288 S Fwy NB &  The S Lp
end: 288 S Fwy NB &  stopped traffic back to Reed Rd. 

S Sam Houston Tollway EB between Hwy 288 and Cullen 
start: S Sam Houston Tollway EB & Hwy 288 
end: S Sam Houston Tollway EB & Cullen 

S Sam Houston Tollway EB between Hwy 288 and Cullen 
start: S Sam Houston Tollway EB & Hwy 288 
end: S Sam Houston Tollway EB & Cullen 

S Sam Houston Tollway EB between Hwy 288 and Cullen 
start: S Sam Houston Tollway EB & Hwy 288 
end: S Sam Houston Tollway EB & Cullen 



In [229]:
# View
output_tweets_df

,city,start,end,tweet
0,"Houston, TX",288 S Fwy NB & The S Lp,288 S Fwy NB & stopped traffic back to Reed Rd.,Closed due to road construction on 288 S Fwy N...
1,"Houston, TX",S Sam Houston Tollway EB & Hwy 288,S Sam Houston Tollway EB & Cullen,Closed due to road construction in #Southside ...
2,"Houston, TX",S Sam Houston Tollway EB & Hwy 288,S Sam Houston Tollway EB & Cullen,Closed due to road construction in #Southside ...
3,"Houston, TX",S Sam Houston Tollway EB & Hwy 288,S Sam Houston Tollway EB & Cullen,Closed due to road construction in #Southside ...


In [242]:
# Final Clean up (maual)
output_tweets_df.loc[0, 'end'] = output_tweets_df.loc[0, 'end'].replace('  stopped traffic back to', '')

#### Final Output

In [249]:
output_tweets_df.drop_duplicates(inplace=True)
output_tweets_df

,city,start,end,tweet
0,"Houston, TX",288 S Fwy NB & The S Lp,288 S Fwy NB & Reed Rd.,Closed due to road construction on 288 S Fwy N...
1,"Houston, TX",S Sam Houston Tollway EB & Hwy 288,S Sam Houston Tollway EB & Cullen,Closed due to road construction in #Southside ...


In [251]:
output_tweets_df.to_csv('../data/3-final/output_tweets', index=False)